In [122]:
import numpy as np
import torch
import pandas as pd
# import numpy as np
from torch import nn
from torch.nn import functional as F




from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score
import xgboost as xgb
from sklearn.metrics import precision_score,recall_score,roc_auc_score,roc_curve
import pickle



def model_load(model, x_transform):
    # 模型加载
    clf = xgb.XGBClassifier()
    booster = xgb.Booster()
    booster.load_model(model)
    clf._Booster = booster

    # 数据预测
    y_pred = [round(value) for value in clf.predict(x_transform)]
    y_pred_proba = clf.predict_proba(x_transform)
    print('y_pred：', y_pred)
    print('y_pred_proba：', y_pred_proba)


In [123]:

# cnn和transformer还挺像的，
# cnn应该代表的是基于词向量（定长输入）的类注意力机制

class SentimentNet(nn.Module):
    def __init__(self, embed_size, num_filter, filter_size, weight, labels, use_gpu, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)

        self.use_gpu = use_gpu
        # 话说这个不会是我前面做的单词到词向量的映射表吧？？ 答：不用怀疑，正是
        self.embedding = nn.Embedding.from_pretrained(weight)
        # 防止嵌入层在训练中更新
        self.embedding.weight.requires_grad = False

        # 啥玩意儿这是？
        self.conv1d = nn.Conv1d(embed_size, num_filter, filter_size, padding=1)
        self.activate = F.relu
        self.decoder = nn.Linear(num_filter, labels)


    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # print(embeddings.shape)
        convolution = self.activate(self.conv1d(embeddings.permute([0, 2, 1])))
        # print(convolution.shape)

        pooling = F.max_pool1d(convolution, kernel_size=convolution.shape[2])

        outputs = self.decoder(pooling.squeeze(dim=2))
        # print(outputs)
        # print(self.decoder)

        return outputs


In [124]:
model_cnn = torch.load('model_cnn_TP_FP.pkl')
# load model from file
model_xgboost = pickle.load(open("xgboost_bert_model.pkl", "rb"))
ids_list = pickle.load(open('TP_FP_ids_list.pkl','rb'))

In [125]:
train = pd.read_csv('bert_train.csv')

def from_data_x():
    data = [eval(str_list) for str_list in train['feature']]
    data = pd.DataFrame(data)
    return data.to_numpy()
def from_data_y():
    labels = train['labels']
    return labels.to_numpy()


# 假设data是包含数据的DataFrame，target是包含标签的Series
X = from_data_x()  # 特征数据
y = from_data_y()  # 标签数据

# 对验证集进行预测——类别
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,shuffle=True)

y_test_xgboost = model_xgboost.predict(X_test)



In [126]:
pickle_file = 'depression_glove_42B_300d_TP_FP.pkl'
[train_features, train_labels, val_features, val_labels, test_features, weight, word_to_idx, idx_to_word, vocab] = pickle.load(open(pickle_file, 'rb'))


X = val_features
y_cnn = val_labels



In [127]:
# 启用推理模式
model_cnn.eval()

# 前向传播
X = X.cuda()

with torch.no_grad():
    output = model_cnn(X)

# 将输出转换为概率
probabilities = torch.nn.functional.softmax(output, dim=1)

# 获取预测结果
_, y_pred_cnn = torch.max(probabilities, 1)


In [128]:
y_pred_cnn = y_pred_cnn.tolist()
y_cnn = y_cnn.tolist()


In [133]:
y_pred_all = y_test_xgboost.tolist().copy()
y_pred_all.extend(y_pred_cnn)
y_all = y_test.tolist().copy()
y_all.extend(y_cnn)


In [134]:
len(y_test)

972

In [135]:
def metrics_sklearn( y_pred_,y_valid):
    """模型对验证集和测试集结果的评分"""
    # 准确率
    accuracy = accuracy_score(y_valid, y_pred_)
    print('Accuracy：%.6f%%' % (accuracy * 100))

    # 精准率
    precision = precision_score(y_valid, y_pred_)
    print('Precision：%.6f%%' % (precision * 100))

    # 召回率
    recall = recall_score(y_valid, y_pred_)
    # print(y_valid)
    # print('-----------------------------')
    # print(y_pred_)
    print('Recall：%.6f%%' % (recall * 100))

    # F1值
    f1 = f1_score(y_valid, y_pred_)
    print('F1：%.6f%%' % (f1 * 100))

    # auc曲线下面积
    auc = roc_auc_score(y_valid, y_pred_)
    print('AUC：%.6f%%' % (auc * 100))

    # ks值
    fpr, tpr, thresholds = roc_curve(y_valid, y_pred_)
    ks = max(abs(fpr - tpr))
    print('KS：%.6f%%' % (ks * 100))


In [136]:
metrics_sklearn(np.array(y_pred_all),np.array(y_all))

Accuracy：84.321224%
Precision：85.555556%
Recall：33.771930%
F1：48.427673%
AUC：66.091344%
KS：32.182688%
